In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np. random.seed(100)

In [ ]:
#  Neuron class = calculation of: output, activation potential, activation functions
class activation_fcn(object):

    def __init__(self):
        pass

    # Claculate neuron output
    def output(self, layer, name, derivative=False):
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, name, "Invalid_function")
        # Call the method as we return it
        return method(layer, derivative)

    # Display error if wrong activation function name is selected
    def Invalid_function(self, *arg):
        print("Error: Invalid activation function")
        return None

    # Identity activation function
    def linear(self, layer, derivative=False):
        print(layer)
        if not derivative:
            return layer
        else:
            return np.ones(shape=np.shape(layer))

    def tanh(self, layer, derivative=False):
        if not derivative:
            return np.tanh(layer)
        else:
            return 1 - np.tanh(layer)**2

    def logistic(self, layer, derivative=False):
        out = 0
        if not derivative:
            out = 1/(1 + np.exp(-layer))
        else:
            out = layer * (1-layer)
        return out

    def relu(self, layer, derivative=False):
        if not derivative:
            return np.maximum(0, layer)
        else:
            return np.where(layer > 0, 1, 0)


In [ ]:
#  Loss function class
class loss_fcn(object):

    def __init__(self):
        pass

    # Loss/error value calculated for all input data sample
    def loss(self, loss, expected, outputs, derivative):
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, str(loss), lambda expected, outputs, derivative: "Invalid loss function")
        # Call the method as we return it

        return method(expected, outputs, derivative)

    # Display error if wrong loss function name is selected
    def Invalid_function(self, *arg):
        print("Error: Invalid loss function")
        return None

    # Mean Square Error loss function
    def mse(self, expected, outputs, derivative=False):
        error_value = 0
        if not derivative:
            error_value = 0.5 * np.power(expected - outputs, 2)
        else:
            error_value = -(expected - outputs)
        return error_value

    # Cross Entorpy loss function
    def binary_cross_entropy(self, expected, outputs, derivative=False):
        error_value = 0
        if not derivative:
            error_value = -(expected * np.log(outputs) + (1 - expected) * np.log(1 - outputs))
        else:
            error_value = -(expected / outputs) + (1 - expected) / (1 - outputs)
        return error_value

In [ ]:
# Initialize a network
class Neural_network(object):

    def __init__(self):
        pass

    def create_network(self, structure, init_weights):
        self.nnetwork = [structure[0]]
        for i in range(1, len(structure)):
            new_layer = {
                'weights': None,
                'bias': structure[i]['bias'],
                'activation_function': structure[i]['activation_function'],
                'activation_potential': None,
                'delta': None,
                'output': None}
            if init_weights == "ones":
                new_layer['weights'] = np.ones((structure[i]['units']))
            elif init_weights == "zeros":
                new_layer['weights'] = np.zeros((structure[i]['units']))
            elif init_weights == "random":
                new_layer['weights'] =  np.random.randn((structure[i]['units']))
            else:
                pass
            self.nnetwork.append(new_layer)
        return self.nnetwork

    # Forward propagate input to a network output
    def forward_propagate(self, nnetwork, inputs):
        nnetwork[0]['output'] = inputs

        act_fn = activation_fcn()
        for i in range(1, len(nnetwork)):
            nnetwork[i]['activation_potential'] = np.sum(np.outer(nnetwork[i-1]['output'], nnetwork[i]['weights']),0) + nnetwork[i]['bias']
            nnetwork[i]['output'] = act_fn.output(nnetwork[i]['activation_potential'], nnetwork[i]['activation_function'])

        return nnetwork[-1]['output']

    # Backpropagate error and store it in neuron
    def backward_propagate(self, loss_function, nnetwork, expected):
        act_fn = activation_fcn()
        loss_fn = loss_fcn()

        # Calculate delta for output layer
        nnetwork[-1]['delta'] = act_fn.output(nnetwork[-1]['output'], nnetwork[-1]['activation_function'], True) * \
                                loss_fn.loss(loss_function, expected, nnetwork[-1]['output'], True)

        # Calculate delta for hidden layers
        for i in range(len(nnetwork)-2, 0, -1):
            nnetwork[i]['delta'] = np.dot(nnetwork[i+1]['weights'], nnetwork[i+1]['delta']) * \
                                   act_fn.output(nnetwork[i]['output'], nnetwork[i]['activation_function'], True)


    # Update network weights with error
    def update_weights(self, nnetwork, inputs, l_rate):
        for i in range(1, len(nnetwork)):
            # Update weights
            nnetwork[i]['weights'] -= l_rate * nnetwork[i]['delta'] * nnetwork[i-1]['output']
            # Update bias
            nnetwork[i]['bias'] -= l_rate * nnetwork[i]['delta']


    # Train a network for a fixed number of epochs
    def train(self, nnetwork, x_train, y_train, l_rate=0.01, n_epoch=100, loss_function='mse', verbose=1):
        for epoch in range(n_epoch):
            sum_error = 0
            for iter, (x_row, y_row) in enumerate(zip(x_train, y_train)):

                self.forward_propagate(nnetwork, x_row)

                loss = loss_fcn()
                sum_error = np.sum(loss.loss(loss_function, y_row, nnetwork[-1]['output'], derivative=False))

                self.backward_propagate(loss_function, nnetwork, y_row)

                self.update_weights(nnetwork, x_row, l_rate)

            if verbose > 0:
                print('>epoch=%d, loss=%.3f' % (epoch + 1, sum_error))
        print('Results: epoch=%d, loss=%.3f' % (epoch + 1, sum_error))
        return nnetwork

    # Calculate network output
    def predict(self, nnetwork, inputs):

        potentials = []
        for i in inputs:
            potentials.append(self.forward_propagate(nnetwork, i))

        return potentials


In [ ]:
def generate_classification_data(n=30):
    # Class 1 - samples generation
    X1_1 = 1 + 4 * np.random.rand(n, 1)
    X1_2 = 1 + 4 * np.random.rand(n, 1)
    class1 = np.concatenate((X1_1, X1_2), axis=1)
    Y1 = np.ones(n)

    # Class 0 - samples generation
    X0_1 = 3 + 4 * np.random.rand(n, 1)
    X0_2 = 3 + 4 * np.random.rand(n, 1)
    class0 = np.concatenate((X0_1, X0_2), axis=1)
    Y0 = np.zeros(n)

    X = np.concatenate((class1, class0))
    Y = np.concatenate((Y1, Y0))

    idx0 = [i for i, v in enumerate(Y) if v == 0]
    idx1 = [i for i, v in enumerate(Y) if v == 1]

    return X, Y, idx0, idx1


def test_classification():
    # Read data
    X, Y, idx0, idx1 = generate_classification_data()

    # Create network
    model = Neural_network()
    structure = [{'type': 'input', 'units': 2},
                 {'type': 'dense', 'units': 4, 'activation_function': 'relu', 'bias': True},
                 {'type': 'dense', 'units': 4, 'activation_function': 'relu', 'bias': True},
                 {'type': 'dense', 'units': 1, 'activation_function': 'logistic', 'bias': True}]

    network = model.create_network(structure, "random")

    model.train(network, X, Y, 0.0001, 2000, 'binary_cross_entropy', 0)

    y = model.predict(network, X)
    t = 0

    for n, m in zip(y, Y):
        t += 1 - np.abs(np.round(np.array(n)) - np.array(m))
        print(f"pred = {n}, pred_round = {np.round(n)}, true = {m}")

    ACC = t / len(X)
    print(f"\nClassification accuracy = {ACC * 100}%")

    # Plotting decision regions
    xx, yy = np.meshgrid(np.arange(0, 8, 0.1),
                         np.arange(0, 8, 0.1))

    X_vis = np.c_[xx.ravel(), yy.ravel()]

    h = model.predict(network, X_vis)
    h = np.array(h) >= 0.5
    h = np.reshape(h, (len(xx), len(yy)))

    plt.contourf(xx, yy, h, cmap='jet')
    plt.scatter(X[idx1, 0], X[idx1, 1], marker='^', c="red", edgecolors="white", label="class 1")
    plt.scatter(X[idx0, 0], X[idx0, 1], marker='o', c="blue", edgecolors="white", label="class 0")
    plt.show()


generate_classification_data()
test_classification()

In [ ]:
def generate_regression_data(n=30):
    # Generate regression dataset
    X = np.linspace(-5, 5, n).reshape(-1, 1)
    y = np.sin(2 * X) + np.cos(X) + 5
    # simulate noise
    data_noise = np.random.normal(0, 0.2, n).reshape(-1, 1)
    # Generate training data
    Y = y + data_noise

    return X.reshape(-1, 1), Y.reshape(-1, 1)


def test_regression():
    # Read data
    X, Y = generate_regression_data()

    # Create network
    model = Neural_network()
    structure = [{'type': 'input', 'units': 1},
                 {'type': 'dense', 'units': 8, 'activation_function': 'tanh', 'bias': True},
                 {'type': 'dense', 'units': 8, 'activation_function': 'tanh', 'bias': True},
                 {'type': 'dense', 'units': 1, 'activation_function': 'linear', 'bias': True}]

    network = model.create_network(structure, 'random')

    model.train(network, X, Y, 0.01, 4000, 'mse', 0)

    predicted = model.predict(network, X)
    std = np.std(predicted - Y)
    print("\nStandard deviation = {}".format(std))

    X_test = np.linspace(-7, 7, 100).reshape(-1, 1)
    X_test = np.array(X_test).tolist()
    predicted = model.predict(network, X_test)

    plt.plot(X, Y, 'r--o', label="Training data")
    plt.plot(X_test, predicted, 'b--x', label="Predicted")
    plt.legend()
    plt.grid()
    plt.show()

generate_regression_data(30)
test_regression()